In [ ]:
%run all_players.ipynb



import nbimporter
from all_players import scrape_player_stats
import pandas as pd
from bs4 import BeautifulSoup, Comment
import requests
import time

leagues = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/schedule/{year}-Bundesliga-Scores-and-Fixtures"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/schedule/{year}-Premier-League-Scores-and-Fixtures"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/schedule/{year}-Serie-A-Scores-and-Fixtures"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/schedule/{year}-La-Liga-Scores-and-Fixtures"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/schedule/{year}-Ligue-1-Scores-and-Fixtures"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/schedule/{year}-Eredivisie-Scores-and-Fixtures"
    },
    {        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/schedule/{year}-Primeira-Liga-Scores-and-Fixtures"
    
    },
    {
        "name": "EFL Championship",
        "url": "https://fbref.com/en/comps/10/{year}/schedule/{year}-EFL-Championship-Scores-and-Fixtures"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/schedule/{year}-Scottish-Premiership-Scores-and-Fixtures"
    }
]


200 https://fbref.com/en/comps/20/2024-2025/stats/2024-2025-Bundesliga-Stats
Found stats_standard in comment for Bundesliga 2024-2025
200 https://fbref.com/en/comps/9/2024-2025/stats/2024-2025-Premier-League-Stats
Found stats_standard in comment for Premier League 2024-2025
200 https://fbref.com/en/comps/12/2024-2025/stats/2024-2025-La-Liga-Stats
Found stats_standard in comment for La Liga 2024-2025
200 https://fbref.com/en/comps/11/2024-2025/stats/2024-2025-Serie-A-Stats
Found stats_standard in comment for Serie A 2024-2025
